# Libraries

In [1]:
import os
import shutil
import ffmpeg

# Helper functions

In [32]:
def copy_videos(source_dir, destination_dir, overwrite=True):
    """Copy 'pre' eCig and Cig videos from source directory to destination directory."""
    os.makedirs(destination_dir, exist_ok=True)

    # Initialize counters
    cig = 0
    eCig = 0
    total = 0

    print("Copying eCig and Cig 'pre' videos.")

    # Validate if source directory exists
    if not os.path.exists(source_dir):
        print(f"Error: Source directory '{source_dir}' does not exist.")
        return

    # Get existing files in the destination directory (for faster lookup)
    existing_files = set(os.listdir(destination_dir))

    # Iterate over files in the source directory
    for video_file in os.listdir(source_dir):
        full_source_path = os.path.join(source_dir, video_file)

        # Count total .mp4 files
        if video_file.endswith(".mp4"):
            total += 1

        # Skip if the file already exists in the destination and `overwrite` is False
        if not(overwrite) and (video_file in existing_files):
            continue

        # Check if it's a valid file and matches the criteria
        if os.path.isfile(full_source_path) and "pre" in video_file.lower():
            if "ecig" in video_file.lower():
                eCig += 1
            elif "cig" in video_file.lower():
                cig += 1
            else:
                continue

            # Copy file to the destination directory
            shutil.copy(full_source_path, destination_dir)

    print(f"Copy complete!\n\t eCig pre: {eCig}, Cig pre: {cig}. Total: {eCig + cig} out of {total}")


def process_video(input_dir, output_dir, video_file, overwrite=True, duration=900):
    """Trim video, apply uniform gamma correction and brightness adjustment, convert to grayscale."""
    
    # Define input and output paths
    input_path = os.path.join(input_dir, video_file)
    output_path = os.path.join(output_dir, video_file)

    # Check if file already exists and skip if overwrite is False
    if not overwrite and os.path.exists(output_path):
        print(f"Skipping {video_file}: already exists in {output_dir}.")
        return

    try:
        # Apply gamma correction, brightness, contrast adjustments, and convert to grayscale
        print("\tApplying gamma correction, brightness normalization, and converting to grayscale.")
        (
            ffmpeg
            .input(input_path, t=duration, hwaccel="cuda")  # Trim video to specified duration and hardware acceleration using the gpu
            .filter('format', 'gray')  # convert video to grayscale
            .filter('eq', gamma=1.8, brightness=0.17, contrast=1.3)  # EQ Adjustments
            .filter('scale', 'iw', 'ih', flags='lanczos')  # Clamp pixel values
            .output(output_path, vcodec="hevc_nvenc")  # nvidia vcodec
            .run(overwrite_output=True)
        )
        print(f"\tProcessing completed -> {output_path}")

    except ffmpeg.Error as e:
        print(f"Error processing {video_file}: {e.stderr.decode()}")
    except Exception as e:
        print(f"Unexpected error processing {video_file}: {e}")

# Data Gathering & Video Preprocessing

In [ ]:
# Copy the right videos from the source directory
source_dir = "/mnt/olimpo1/neurocig/data"
destination_dir = "/home/jalal/projects/neurocig/data/vids/raw"

copy_videos(source_dir, destination_dir, overwrite=False)

In [ ]:
# Video pre-processing 
input_dir = "/home/jalal/projects/neurocig/data/vids/processed/test"
output_dir = "/home/jalal/projects/neurocig/data/vids/processed/test/hist_tk"
os.makedirs(output_dir, exist_ok=True)


for video_file in os.listdir(input_dir):
    if video_file.endswith('.mp4'):
        process_video(input_dir, output_dir, video_file)